In [1]:
import pandas as pd
import os

In [2]:
bookings = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-t-varenichenko-22/My_practice/Modul_1_Python/Lesson_2/bookings.csv', sep = ';')

In [3]:
bookings.head(5)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [4]:
bookings.shape

(119390, 21)

In [5]:
bookings.columns

Index(['Hotel', 'Is Canceled', 'Lead Time', 'arrival full date',
       'Arrival Date Year', 'Arrival Date Month', 'Arrival Date Week Number',
       'Arrival Date Day of Month', 'Stays in Weekend nights',
       'Stays in week nights', 'stays total nights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'Reserved Room Type', 'Assigned room type',
       'customer type', 'Reservation Status', 'Reservation status_date'],
      dtype='object')

In [6]:
books = bookings.rename(columns = {'Assigned room type': 'Assigned_room_type', 'Reserved Room Type' : 'Reserved_Room_Type',
                                   'Arrival Date Year': 'arrival_date_year', 
                                  'Arrival Date Month' : 'arrival_date_month',
                                  'Is Canceled': 'is_canceled'})

In [7]:
bookings[['stays total nights', 'Hotel']].groupby('Hotel', as_index = False).aggregate({'stays total nights':'mean'}).round(2)

,Hotel,stays total nights
0,City Hotel,2.98
1,Resort Hotel,4.32


Иногда тип номера, присвоенного клиенту (assigned_room_type), отличается от изначально забронированного (reserved_room_type).
Такое может произойти, например, по причине овербукинга.
Сколько подобных наблюдений встретилось в датасете?
*отмена бронирования также считается

In [8]:
books.query('Assigned_room_type != Reserved_Room_Type').shape

(14917, 21)

На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017?

In [9]:
books.query('arrival_date_year == 2017')['arrival_date_month'].value_counts()

May         6313
April       5661
June        5647
July        5313
March       4970
August      4925
February    4177
January     3681
Name: arrival_date_month, dtype: int64

На какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015 - 2017 гг.?

In [11]:
book = books.query("Hotel == 'City Hotel' & is_canceled == 1")[['arrival_date_year','arrival_date_month']]
book.value_counts()

arrival_date_year  arrival_date_month
2017               May                   2217
2016               October               1947
2017               April                 1926
                   June                  1808
2016               June                  1720
                   September             1567
2015               September             1543
2016               April                 1539
                   May                   1436
                   November              1360
2017               July                  1324
2015               October               1321
2017               March                 1278
2016               August                1247
2015               August                1232
2017               August                1123
2016               March                 1108
                   December              1072
2017               January               1044
2016               July                  1043
2017               February               

In [12]:
books.query("Hotel == 'City Hotel' & is_canceled == 1").groupby('arrival_date_year')['arrival_date_month'].value_counts()

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

In [13]:
#Посмотрите на числовые характеристики трёх колонок: adults, children и babies. 
#Какая из них имеет наибольшее среднее значение?#

bookings[['Adults', 'Children', 'Babies']].mean().round(2)

Adults      1.86
Children    0.10
Babies      0.01
dtype: float64

In [14]:
# Создайте колонку total_kids, объединив столбцы children и babies. #
# Для отелей какого типа среднее значение переменной оказалось наибольшим?#
# City hotel – отель находится в городе#
# Resort hotel – отель курортный#
# В качестве ответа укажите наибольшее среднее total_kids, округлив до 2 знаков после точки.#

bookings['total_kids'] = bookings.Children + bookings.Babies
bookings.groupby('Hotel')\
        .agg({'total_kids': 'mean'}) \
        .round(2).max()

total_kids    0.14
dtype: float64

In [15]:
#Исправьте ситуацию, приведя названия столбцов к нижнему регистру и заменив пробелы на знак нижнего подчеркивания. #
def replace_space_with_(name):
    new_name = name.replace(' ', '_').lower()    
    return new_name
bookings = bookings.rename(columns = replace_space_with_)
bookings.head(3)

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0


In [18]:
#Пользователи из каких стран совершили наибольшее число успешных бронирований?# 
#Бронирование считается успешным, если в дальнейшем не было отменено (переменная is_canceled).#
#В качестве ответа выберите страны, входящие в топ-5.#
bookings.query("is_canceled == '0'")['country'].value_counts()

Series([], Name: country, dtype: int64)

Посмотрим на Churn Rate, как эта метрика связана с наличием детей у клиентов

In [19]:
bookings['total_kids'] = bookings.children + bookings.babies
bookings['has_kids'] = bookings['total_kids'] > 0
no_kids_Churn = bookings.query('is_canceled == 1 and has_kids == False').shape[0]/bookings.query('has_kids == False').shape[0]
round(no_kids_Churn*100,2)

37.22

In [20]:
kids_Churn = bookings.query('is_canceled == 1 and has_kids == True').shape[0]/bookings.query('has_kids == True').shape[0]
round(kids_Churn*100,2)

34.92